# A simple bottleneck réalisé par Danial HABIB et Maroun CHAHINE

Very simple code that :
- make_random_edges : generate a random graph given a number of edge and nodes
- find_unique_edges : filter the list of edges to only keep an instance of each (i.e. remove duplicate edges)
- contains : verify if an edge is part of a list

In [47]:
import random

def make_random_edges(n_edges=10000, n_nodes=10):
    random.seed(42)
    edges = [[random.randint(0, n_nodes), random.randint(0, n_nodes)] for e in range(n_edges)]
    return edges

def find_unique_edges(edges):
    edges = list(edges)
    unique_edges = []
    while edges:
        edge = edges.pop()
        if not contains(edges, edge):
            unique_edges.append(edge)
    return unique_edges

def contains(edges, edge):
    for e in edges:
        if sorted(e) == sorted(edge):
            return True
    return False


print("Generating random edges...")
edges = make_random_edges()
print(f"Total edges generated: {len(edges)}")
print(edges)
print("Finding unique edges...")
unique_edges = find_unique_edges(edges)
print(f"Total unique edges found: {len(unique_edges)}")
print(unique_edges)

Generating random edges...
Total edges generated: 10000
[[10, 1], [0, 4], [3, 3], [2, 1], [10, 8], [1, 9], [6, 0], [0, 1], [3, 3], [8, 9], [0, 8], [3, 10], [8, 6], [3, 7], [9, 4], [0, 2], [6, 5], [4, 2], [3, 5], [1, 1], [6, 1], [5, 5], [9, 4], [0, 7], [8, 1], [6, 1], [8, 4], [10, 9], [5, 9], [3, 1], [0, 10], [3, 4], [1, 3], [1, 6], [4, 7], [10, 5], [2, 5], [5, 3], [10, 4], [10, 10], [1, 9], [10, 2], [8, 3], [2, 7], [6, 4], [10, 8], [3, 10], [5, 0], [3, 0], [5, 6], [4, 1], [3, 9], [5, 3], [10, 7], [6, 10], [7, 2], [4, 2], [3, 8], [8, 4], [9, 6], [9, 6], [5, 3], [2, 8], [7, 1], [0, 1], [2, 10], [2, 10], [6, 9], [1, 6], [6, 9], [7, 8], [4, 8], [0, 10], [1, 10], [8, 4], [10, 5], [1, 4], [6, 2], [7, 0], [4, 8], [2, 8], [1, 10], [4, 10], [8, 9], [3, 2], [5, 2], [8, 8], [0, 9], [5, 7], [0, 1], [5, 4], [3, 0], [3, 9], [1, 1], [7, 1], [8, 2], [2, 10], [7, 8], [2, 4], [8, 9], [6, 3], [8, 3], [4, 6], [10, 10], [5, 7], [8, 7], [1, 3], [3, 1], [5, 0], [9, 8], [3, 9], [3, 0], [1, 10], [0, 3], [1, 0]

Some profiling functions to ease the process

In [28]:
from functools import wraps
from cProfile import Profile
from tempfile import NamedTemporaryFile
import pstats

_time_profiles = {}

def profile_time(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        profile = Profile()
        ret = profile.runcall(func, *args, **kwargs)
        _time_profiles[(wrapper, ) + args] = profile
        
        return ret 
    return wrapper

def profile_stats(profile):
    temp_stats = NamedTemporaryFile()
    profile.dump_stats(temp_stats.name)
    return pstats.Stats(temp_stats.name)

Pour utiliser la wrapper function `@profile_time`, il suffit de le placer juste au-dessus de la définition de la fonction que vous souhaitez profiler. Cela permet automatiquement de mesurer le temps d'exécution de la fonction et stocker les résultats dans `_time_profiles`.

Simple profiling with text outputs

In [34]:
@profile_time
def remove_duplicate_edges(n_edges=2000):
    edges = make_random_edges(n_edges, 200)
    unique_edges = find_unique_edges(edges)
    return unique_edges

unique_edges = remove_duplicate_edges(2000)

profile = _time_profiles[(remove_duplicate_edges, 2000)]

stats = profile_stats(profile) # convert raw profile data to pstats readable format
stats.strip_dirs() # remove extraneous path from file names
stats.sort_stats('time') # sort by internal time spent in each function, slowest first
stats.print_stats() # print the stats to stdout

Fri Dec  5 16:55:08 2025    /tmp/tmpd48vt5on

         3942601 function calls in 1.325 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
  3903622    0.714    0.000    0.714    0.000 {built-in method builtins.sorted}
     2000    0.596    0.000    1.310    0.001 3010000368.py:17(contains)
     4000    0.004    0.000    0.008    0.000 random.py:292(randrange)
     4000    0.003    0.000    0.011    0.000 random.py:366(randint)
     4000    0.003    0.000    0.004    0.000 random.py:239(_randbelow_with_getrandbits)
        1    0.002    0.002    1.313    1.313 3010000368.py:8(find_unique_edges)
        1    0.001    0.001    0.012    0.012 3010000368.py:5(<listcomp>)
    12000    0.001    0.000    0.001    0.000 {built-in method _operator.index}
     5051    0.001    0.000    0.001    0.000 {method 'getrandbits' of '_random.Random' objects}
     4000    0.000    0.000    0.000    0.000 {method 'bit_length' of 'int' objects}
   

Le profiling nous donne des statistiques sur l'exécution d'une fonction. Par exemple, pour l'appel `remove_duplicate_edges(2000)`, on obtient le nombre d'appels de chaque fonction (ex: 3903622 appels à `sorted()` et 4000 à `randint()`), le temps total passé dans chaque fonction (ex: 1.036 seconde dans `find_unique_edges()`), et le temps par appel. Ces informations permettent d'identifier rapidement les fonctions qui consomment le plus de temps et qui nécessitent une optimisation.

Advance profiling with visualization

In [ ]:
%reload_ext snakeviz

In [ ]:
from snakeviz.ipymagic import open_snakeviz_and_display_in_notebook

def display_stats(profile):
    temp_stats = NamedTemporaryFile(delete=False)
    profile.dump_stats(temp_stats.name)
    return open_snakeviz_and_display_in_notebook(temp_stats.name)

In [ ]:
@profile_time
def remove_duplicate_edges(n_edges=2000):
    edges = make_random_edges(n_edges, 200)
    unique_edges = find_unique_edges(edges)
    return unique_edges

unique_edges = remove_duplicate_edges(2000)

profile = _time_profiles[(remove_duplicate_edges, 2000)]
#display_stats(profile)

# Save profile to a file and open with snakeviz in browser
import os
profile_file = '/tmp/profile_output.prof'
profile.dump_stats(profile_file)
print(f"Profile saved to: {profile_file}")
print("\nTo view in browser, run this command in a terminal:")
print(f"snakeviz {profile_file}")
print("\nOr run this to open automatically:")
os.system(f"snakeviz {profile_file} &")

Profile saved to: /tmp/profile_output.prof

To view in browser, run this command in a terminal:
snakeviz /tmp/profile_output.prof

Or run this to open automatically:


0

Port 8080 in use, trying another.
snakeviz web server started on 127.0.0.1:8081; enter Ctrl-C to exit
http://127.0.0.1:8081/snakeviz/%2Ftmp%2Fprofile_output.prof


On obtient les mêmes résultats, mais cette fois avec une approche graphique.

Propose a new implementation and profile it

Nous allons à présent proposer une nouvelle implémentation qui réalise le même but que l'algorithme initial. L'algorithme doit donc créer une liste de sommets, puis enlever tous les doublons. Dans cette implémentation, nous allons mettre en du parallellislme massif dans le but de reduire le temps d'exécution de l'algorithme. Nous allons également profiler le nouvel algo pour pouvoir comparer les deux implémentations en terme de temps d'exécution.

Nous allons d'abord utiliser l'algo de tri en parallèle que nous avons implémenté dans la partie précédente "complexity" à la place de l'algo sorted() de Python. Nous avons vu précedemment qu'il est plus optimisé pour de grandes listes.

In [30]:
import numba as numba


@numba.njit
def insertion_sort_numba(ar):
    ar = ar.copy()
    n = len(ar)
    for i in numba.prange(1, n):
        key_item = ar[i]
        j = i - 1
        while j >= 0 and ar[j] > key_item:
            ar[j + 1] = ar[j]
            j -= 1
        ar[j + 1] = key_item
    return ar

def make_random_edges_hpc(n_edges=100, n_nodes=1000):
    random.seed(42)
    edges = [[random.randint(0, n_nodes), random.randint(0, n_nodes)] for e in range(n_edges)]
    return edges

def find_unique_edges_hpc(edges):
    edges = list(edges)
    unique_edges = []
    while edges:
        edge = edges.pop()
        if not contains_hpc(edges, edge):
            unique_edges.append(edge)
    return unique_edges

def contains_hpc(edges, edge):
    for e in edges:
        if insertion_sort_numba(e) == insertion_sort_numba(edge):
            return True
    return False


print("Generating random edges...")
edges = make_random_edges_hpc()
print(f"Total edges generated: {len(edges)}")
print(edges)
print("Finding unique edges...")
unique_edges = find_unique_edges_hpc(edges)
print(f"Total unique edges found: {len(unique_edges)}")
print(unique_edges)

Generating random edges...
Total edges generated: 100
[[654, 114], [25, 759], [281, 250], [228, 142], [754, 104], [692, 758], [913, 558], [89, 604], [432, 32], [30, 95], [223, 238], [517, 616], [27, 574], [203, 733], [665, 718], [558, 429], [225, 459], [603, 284], [828, 890], [6, 777], [825, 163], [714, 432], [348, 284], [159, 220], [980, 781], [344, 104], [94, 389], [99, 367], [867, 352], [618, 270], [826, 44], [747, 470], [549, 127], [996, 944], [387, 80], [565, 300], [849, 643], [633, 906], [882, 370], [591, 196], [721, 71], [46, 677], [233, 791], [296, 81], [875, 238], [887, 103], [389, 284], [464, 650], [854, 373], [166, 379], [363, 214], [686, 273], [718, 959], [699, 663], [73, 623], [650, 175], [546, 746], [250, 167], [473, 388], [276, 947], [655, 704], [570, 224], [701, 332], [863, 786], [794, 57], [234, 841], [32, 824], [323, 410], [274, 67], [216, 935], [965, 580], [897, 735], [322, 217], [671, 511], [405, 905], [936, 658], [469, 146], [271, 142], [252, 762], [574, 551], [269

In [33]:
@profile_time
def remove_duplicate_edges_hpc(n_edges=2000):
    edges = make_random_edges_hpc(n_edges, 20)
    unique_edges = find_unique_edges_hpc(edges)
    return unique_edges

unique_edges = remove_duplicate_edges_hpc(2000)

profile = _time_profiles[(remove_duplicate_edges_hpc, 2000)]

stats = profile_stats(profile) # convert raw profile data to pstats readable format
stats.strip_dirs() # remove extraneous path from file names
stats.sort_stats('time') # sort by internal time spent in each function, slowest first
stats.print_stats() # print the stats to stdout

Fri Dec  5 16:49:41 2025    /tmp/tmp9x_40vso

         29922716 function calls in 10.721 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
   807686    1.468    0.000    9.793    0.000 152500172.py:4(insertion_sort_numba)
  2423058    1.385    0.000    8.325    0.000 typeof.py:27(typeof)
  2423058    1.247    0.000    1.969    0.000 utils.py:464(bit_length)
  2423058    1.209    0.000    6.009    0.000 functools.py:884(wrapper)
  2423058    1.106    0.000    1.906    0.000 functools.py:818(dispatch)
  2423058    0.924    0.000    2.893    0.000 typeof.py:128(_typeof_int)
     2000    0.914    0.000   10.707    0.005 152500172.py:31(contains_hpc)
  2423058    0.589    0.000    0.932    0.000 <string>:1(<lambda>)
  2423058    0.585    0.000    0.585    0.000 weakref.py:415(__getitem__)
  2423058    0.343    0.000    0.343    0.000 {built-in method __new__ of type object at 0x5608933b49a0}
  2423058    0.281    0.000    0.281   

On voit bien que le tri se fait sur des listes de 2 éléments.

`insertion_sort_numba()` est plus lent que `sorted()`. Les deux fonctions créent une nouvelle copie triée sans modifier l'original. Cependant, pour des listes de seulement 2 éléments, le temps de compilation de Numba coûte plus cher que le gain de performance. C'est ce qu'on a vu dans la partie complexity.

Cela a montré  qu'utiliser le sort de numba n'a pas réduit le coût car ce tri n'est pas adéquat dans notre cas, pour de petites listes.

Nous allons essayé une meilleure approche qui utilise les sets et qui a une meilleur complexité.

In [49]:
def make_random_edges_optimized(n_edges=10000, n_nodes=1000):
    random.seed(42)
    edges = [[random.randint(0, n_nodes), random.randint(0, n_nodes)] for e in range(n_edges)]
    return edges

def find_unique_edges_optimized(edges):
    seen = set()
    unique_edges = []
    
    for edge in edges:
        normalized = tuple(sorted(edge))
        if normalized not in seen:
            seen.add(normalized)
            unique_edges.append(edge)
    
    return unique_edges


print("Generating random edges (optimized version)...")
edges = make_random_edges_optimized()
print(f"Total edges generated: {len(edges)}")
print(edges)
print("Finding unique edges...")
unique_edges = find_unique_edges_optimized(edges)
print(f"Total unique edges found: {len(unique_edges)}")
print(unique_edges)

Generating random edges (optimized version)...
Total edges generated: 10000
[[654, 114], [25, 759], [281, 250], [228, 142], [754, 104], [692, 758], [913, 558], [89, 604], [432, 32], [30, 95], [223, 238], [517, 616], [27, 574], [203, 733], [665, 718], [558, 429], [225, 459], [603, 284], [828, 890], [6, 777], [825, 163], [714, 432], [348, 284], [159, 220], [980, 781], [344, 104], [94, 389], [99, 367], [867, 352], [618, 270], [826, 44], [747, 470], [549, 127], [996, 944], [387, 80], [565, 300], [849, 643], [633, 906], [882, 370], [591, 196], [721, 71], [46, 677], [233, 791], [296, 81], [875, 238], [887, 103], [389, 284], [464, 650], [854, 373], [166, 379], [363, 214], [686, 273], [718, 959], [699, 663], [73, 623], [650, 175], [546, 746], [250, 167], [473, 388], [276, 947], [655, 704], [570, 224], [701, 332], [863, 786], [794, 57], [234, 841], [32, 824], [323, 410], [274, 67], [216, 935], [965, 580], [897, 735], [322, 217], [671, 511], [405, 905], [936, 658], [469, 146], [271, 142], [252, 

In [50]:
@profile_time
def remove_duplicate_edges_optimized(n_edges=2000):
    edges = make_random_edges_optimized(n_edges, 200)
    unique_edges = find_unique_edges_optimized(edges)
    return unique_edges

unique_edges = remove_duplicate_edges_optimized(2000)

profile = _time_profiles[(remove_duplicate_edges_optimized, 2000)]

stats = profile_stats(profile)
stats.strip_dirs()
stats.sort_stats('time')
stats.print_stats()

Fri Dec  5 17:04:06 2025    /tmp/tmp7udgrsjl

         38898 function calls in 0.012 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
     4000    0.005    0.000    0.009    0.000 random.py:292(randrange)
     4000    0.002    0.000    0.003    0.000 random.py:239(_randbelow_with_getrandbits)
        1    0.001    0.001    0.002    0.002 1031118430.py:6(find_unique_edges_optimized)
    12000    0.001    0.000    0.001    0.000 {built-in method _operator.index}
        1    0.001    0.001    0.011    0.011 1031118430.py:3(<listcomp>)
     4000    0.001    0.000    0.010    0.000 random.py:366(randint)
     5051    0.001    0.000    0.001    0.000 {method 'getrandbits' of '_random.Random' objects}
     2000    0.000    0.000    0.000    0.000 {built-in method builtins.sorted}
     4000    0.000    0.000    0.000    0.000 {method 'bit_length' of 'int' objects}
     1919    0.000    0.000    0.000    0.000 {method 'add' of 'set'

On est passé d'un algo ayant une complexité O(n2) à à algo de complexité O(n), beaucoup plus performant.
On parcourt la liste une seule fois (O(n)) et la recherche dans un set est O(1) au lieu de O(n)

Nous allons maintenat paralléliser cet algo.

## Version parallèle avec Numba



In [54]:
import numpy as np

@numba.njit(parallel=True)
def normalize_edges_parallel(edges):
    n = len(edges)
    normalized = np.empty((n, 2), dtype=np.int64)
    
    for i in numba.prange(n):  # Parallélisation sur les edges
        if edges[i, 0] < edges[i, 1]:
            normalized[i, 0] = edges[i, 0]
            normalized[i, 1] = edges[i, 1]
        else:
            normalized[i, 0] = edges[i, 1]
            normalized[i, 1] = edges[i, 0]
    
    return normalized

def make_random_edges_parallel(n_edges=10000, n_nodes=10):
    random.seed(42)
    edges = [[random.randint(0, n_nodes), random.randint(0, n_nodes)] for e in range(n_edges)]
    return edges

def find_unique_edges_parallel(edges):
    edges = np.array(edges, dtype=np.int64)
    normalized = normalize_edges_parallel(edges)
    
    seen = set()
    unique_edges = []
    
    for i in range(len(normalized)):
        edge_tuple = (normalized[i, 0], normalized[i, 1])
        if edge_tuple not in seen:
            seen.add(edge_tuple)
            unique_edges.append(edges[i].tolist())
    
    return unique_edges


print("Generating random edges (parallel version)...")
edges = make_random_edges_parallel()
print(f"Total edges generated: {len(edges)}")
print(edges)
print("Finding unique edges...")
unique_edges = find_unique_edges_parallel(edges)
print(f"Total unique edges found: {len(unique_edges)}")
print(unique_edges)

Generating random edges (parallel version)...
Total edges generated: 10000
[[10, 1], [0, 4], [3, 3], [2, 1], [10, 8], [1, 9], [6, 0], [0, 1], [3, 3], [8, 9], [0, 8], [3, 10], [8, 6], [3, 7], [9, 4], [0, 2], [6, 5], [4, 2], [3, 5], [1, 1], [6, 1], [5, 5], [9, 4], [0, 7], [8, 1], [6, 1], [8, 4], [10, 9], [5, 9], [3, 1], [0, 10], [3, 4], [1, 3], [1, 6], [4, 7], [10, 5], [2, 5], [5, 3], [10, 4], [10, 10], [1, 9], [10, 2], [8, 3], [2, 7], [6, 4], [10, 8], [3, 10], [5, 0], [3, 0], [5, 6], [4, 1], [3, 9], [5, 3], [10, 7], [6, 10], [7, 2], [4, 2], [3, 8], [8, 4], [9, 6], [9, 6], [5, 3], [2, 8], [7, 1], [0, 1], [2, 10], [2, 10], [6, 9], [1, 6], [6, 9], [7, 8], [4, 8], [0, 10], [1, 10], [8, 4], [10, 5], [1, 4], [6, 2], [7, 0], [4, 8], [2, 8], [1, 10], [4, 10], [8, 9], [3, 2], [5, 2], [8, 8], [0, 9], [5, 7], [0, 1], [5, 4], [3, 0], [3, 9], [1, 1], [7, 1], [8, 2], [2, 10], [7, 8], [2, 4], [8, 9], [6, 3], [8, 3], [4, 6], [10, 10], [5, 7], [8, 7], [1, 3], [3, 1], [5, 0], [9, 8], [3, 9], [3, 0], [1, 

In [55]:
@profile_time
def remove_duplicate_edges_parallel(n_edges=2000):
    edges = make_random_edges_parallel(n_edges, 200)
    unique_edges = find_unique_edges_parallel(edges)
    return unique_edges

unique_edges = remove_duplicate_edges_parallel(2000)

profile = _time_profiles[(remove_duplicate_edges_parallel, 2000)]

stats = profile_stats(profile)
stats.strip_dirs()
stats.sort_stats('time')
stats.print_stats()

Fri Dec  5 17:06:44 2025    /tmp/tmpxqv1f8os

         38821 function calls in 0.025 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.006    0.006    0.015    0.015 1071080393.py:23(find_unique_edges_parallel)
     4000    0.005    0.000    0.008    0.000 random.py:292(randrange)
     1919    0.004    0.000    0.004    0.000 {method 'tolist' of 'numpy.ndarray' objects}
        1    0.003    0.003    0.003    0.003 1071080393.py:3(normalize_edges_parallel)
     4000    0.002    0.000    0.003    0.000 random.py:239(_randbelow_with_getrandbits)
        1    0.001    0.001    0.001    0.001 {built-in method numpy.array}
        1    0.001    0.001    0.010    0.010 1071080393.py:20(<listcomp>)
     4000    0.001    0.000    0.009    0.000 random.py:366(randint)
    12000    0.001    0.000    0.001    0.000 {built-in method _operator.index}
     5051    0.000    0.000    0.000    0.000 {method 'getrandbits' of '_r

Nous avons bien parallélisé l'algorithme en utilisant Numba. Cela permet de traiter plusieurs edges simultanément sur différents cœurs du CPU, ce qui améliore les performances par rapport à la version séquentielle O(n).

Dans le profiling, en comparant avec la première version, les fonctions sont appelées beaucoup moins. Ceci est cohérent car nous avons à la fois réduit la complexité et paralléliser l'algo.